In [1]:
# import standard python libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os, subprocess

# Import python package for working with cooler files and tools for analysis
import cooler
import cooltools.lib.plotting
import cooltools

from packaging import version
if version.parse(cooltools.__version__) < version.parse('0.5.1'):
    raise AssertionError("tutorials rely on cooltools version 0.5.1 or higher,"+
                         "please check your cooltools version and update to the latest")

In [2]:
import warnings
from cytoolz import merge

def saddleplot(
    track,
    saddledata,
    n_bins,
    vrange=None,
    qrange=(0.0, 1.0),
    cmap="coolwarm",
    scale="log",
    vmin=0.5,
    vmax=2,
    color=None,
    title=None,
    xlabel=None,
    ylabel=None,
    clabel=None,
    fig=None,
    fig_kws=None,
    heatmap_kws=None,
    margin_kws=None,
    cbar_kws=None,
    subplot_spec=None,
):
    """
    Generate a saddle plot.
    Parameters
    ----------
    track : pd.DataFrame
        See cooltools.digitize() for details.
    saddledata : 2D array-like
        Saddle matrix produced by `make_saddle`. It will include 2 flanking
        rows/columns for outlier signal values, thus the shape should be
        `(n+2, n+2)`.
    cmap : str or matplotlib colormap
        Colormap to use for plotting the saddle heatmap
    scale : str
        Color scaling to use for plotting the saddle heatmap: log or linear
    vmin, vmax : float
        Value limits for coloring the saddle heatmap
    color : matplotlib color value
        Face color for margin bar plots
    fig : matplotlib Figure, optional
        Specified figure to plot on. A new figure is created if none is
        provided.
    fig_kws : dict, optional
        Passed on to `plt.Figure()`
    heatmap_kws : dict, optional
        Passed on to `ax.imshow()`
    margin_kws : dict, optional
        Passed on to `ax.bar()` and `ax.barh()`
    cbar_kws : dict, optional
        Passed on to `plt.colorbar()`
    subplot_spec : GridSpec object
        Specify a subregion of a figure to using a GridSpec.
    Returns
    -------
    Dictionary of axes objects.
    """

#     warnings.warn(
#         "Generating a saddleplot will be deprecated in future versions, "
#         + "please see https://github.com/open2c_examples for examples on how to plot saddles.",
#         DeprecationWarning,
#     )

    from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
    from matplotlib.colors import Normalize, LogNorm
    from matplotlib import ticker
    import matplotlib.pyplot as plt

    class MinOneMaxFormatter(ticker.LogFormatter):
        def set_locs(self, locs=None):
            self._sublabels = set([vmin % 10 * 10, vmax % 10, 1])

        def __call__(self, x, pos=None):
            if x not in [vmin, 1, vmax]:
                return ""
            else:
                return "{x:g}".format(x=x)

    track_value_col = track.columns[3]
    track_values = track[track_value_col].values

    digitized_track, binedges = cooltools.digitize(
        track, n_bins, vrange=vrange, qrange=qrange
    )
    x = digitized_track[digitized_track.columns[3]].values.astype(int).copy()
    x = x[(x > -1) & (x < len(binedges) + 1)]
    
    # Old version
    # hist = np.bincount(x, minlength=len(binedges) + 1)

    groupmean = track[track.columns[3]].groupby(digitized_track[digitized_track.columns[3]]).mean()
    
    if qrange is not None:
        lo, hi = qrange
        binedges = np.linspace(lo, hi, n_bins + 1)
    
    # Barplot of mean values and saddledata are flanked by outlier bins
    n = saddledata.shape[0]
    X, Y = np.meshgrid(binedges, binedges)
    C = saddledata
    if (n - n_bins) == 2:
        C = C[1:-1, 1:-1]
        groupmean = groupmean[1:-1]

    # Layout
    if subplot_spec is not None:
        GridSpec = partial(GridSpecFromSubplotSpec, subplot_spec=subplot_spec)
    grid = {}
    gs = GridSpec(
        nrows=3,
        ncols=3,
        width_ratios=[0.2, 1, 0.1],
        height_ratios=[0.2, 1, 0.1],
        wspace=0.05,
        hspace=0.05,
    )

    # Figure
    if fig is None:
        fig_kws_default = dict(figsize=(5, 5))
        fig_kws = merge(fig_kws_default, fig_kws if fig_kws is not None else {})
        fig = plt.figure(**fig_kws)

    # Heatmap
    if scale == "log":
        norm = LogNorm(vmin=vmin, vmax=vmax)
    elif scale == "linear":
        norm = Normalize(vmin=vmin, vmax=vmax)
    else:
        raise ValueError("Only linear and log color scaling is supported")

    grid["ax_heatmap"] = ax = plt.subplot(gs[4])
    heatmap_kws_default = dict(cmap="coolwarm", rasterized=True)
    heatmap_kws = merge(
        heatmap_kws_default, heatmap_kws if heatmap_kws is not None else {}
    )
    img = ax.pcolormesh(X, Y, C, norm=norm, **heatmap_kws)
    plt.gca().yaxis.set_visible(False)

    # Margins
    margin_kws_default = dict(edgecolor="k", facecolor=color, linewidth=1)
    margin_kws = merge(margin_kws_default, margin_kws if margin_kws is not None else {})
    # left margin hist
    grid["ax_margin_y"] = plt.subplot(gs[3], sharey=grid["ax_heatmap"])
    
    plt.barh(
        binedges, height=1/len(binedges), width=groupmean, align="edge", **margin_kws
    )
    
    plt.xlim(plt.xlim()[1], plt.xlim()[0])  # fliplr
    plt.ylim(hi, lo)
    plt.gca().spines["top"].set_visible(False)
    plt.gca().spines["bottom"].set_visible(False)
    plt.gca().spines["left"].set_visible(False)
    plt.gca().xaxis.set_visible(False)
    # top margin hist
    grid["ax_margin_x"] = plt.subplot(gs[1], sharex=grid["ax_heatmap"])
    
    plt.bar(
        binedges, width=1/len(binedges), height=groupmean, align="edge", **margin_kws
    )
    
    plt.xlim(lo, hi)
    # plt.ylim(plt.ylim())  # correct
    plt.gca().spines["top"].set_visible(False)
    plt.gca().spines["right"].set_visible(False)
    plt.gca().spines["left"].set_visible(False)
    plt.gca().xaxis.set_visible(False)
    plt.gca().yaxis.set_visible(False)

    # Colorbar
    grid["ax_cbar"] = plt.subplot(gs[5])
    cbar_kws_default = dict(fraction=0.8, label=clabel or "")
    cbar_kws = merge(cbar_kws_default, cbar_kws if cbar_kws is not None else {})
    if scale == "linear" and vmin is not None and vmax is not None:
        grid["cbar"] = cb = plt.colorbar(img, **cbar_kws)
        # cb.set_ticks(np.arange(vmin, vmax + 0.001, 0.5))
        # # do linspace between vmin and vmax of 5 segments and trunc to 1 decimal:
        decimal = 10
        nsegments = 5
        cd_ticks = np.trunc(np.linspace(vmin, vmax, nsegments) * decimal) / decimal
        cb.set_ticks(cd_ticks)
    else:
        grid["cbar"] = cb = plt.colorbar(img, format=MinOneMaxFormatter(), **cbar_kws)
        cb.ax.yaxis.set_minor_formatter(MinOneMaxFormatter())

    # extra settings
    grid["ax_heatmap"].set_xlim(lo, hi)
    grid["ax_heatmap"].set_ylim(hi, lo)
    plt.grid(False)
    plt.axis("off")
    if title is not None:
        grid["ax_margin_x"].set_title(title)
    if xlabel is not None:
        grid["ax_heatmap"].set_xlabel(xlabel)
    if ylabel is not None:
        grid["ax_margin_y"].set_ylabel(ylabel)

    return grid

In [3]:
clr = cooler.Cooler('../cool_input/mlei_10252Cload.cool')  # 20K bins per genome of Mnemiopsis leidyi

## fasta sequence is required for calculating binned profile of GC conent
import bioframe
bins = clr.bins()[:]
genome = bioframe.load_fasta('../data/genome/Mlei_gDNA.fasta');

## note the next command may require installing pysam
gc_cov = bioframe.frac_gc(bins[['chrom', 'start', 'end']], genome)
display(gc_cov)

,chrom,start,end,GC
0,chr_1,0,5126,0.391533
1,chr_1,5126,10252,0.424698
2,chr_1,10252,15378,0.378268
3,chr_1,15378,20504,0.398453
4,chr_1,20504,25630,0.372037
...,...,...,...,...
40002,chr_13,11641146,11646272,0.224346
40003,chr_13,11646272,11651398,0.410066
40004,chr_13,11651398,11656524,0.436403
40005,chr_13,11656524,11661650,0.521264


In [4]:
view_df = pd.DataFrame({'chrom': clr.chromnames,
                        'start': 0,
                        'end': clr.chromsizes.values,
                        'name': clr.chromnames}
                      )
display(view_df)

,chrom,start,end,name
0,chr_1,0,11404665,chr_1
1,chr_2,0,13041538,chr_2
2,chr_3,0,15865815,chr_3
3,chr_4,0,14925576,chr_4
4,chr_5,0,17370771,chr_5
5,chr_6,0,21511028,chr_6
6,chr_7,0,18526857,chr_7
7,chr_8,0,20866784,chr_8
8,chr_9,0,17080198,chr_9
9,chr_10,0,12565900,chr_10


In [5]:
# obtain first 3 eigenvectors
cis_eigs = cooltools.eigs_cis(
                        clr,
                        gc_cov,
                        view_df=view_df,
                        n_eigs=3,
                        )

eigenvector_track = cis_eigs[1][['chrom','start','end', 'E1', 'E2','E3']]
eigenvector_track.to_csv('../compartmentalization/eigenvalues/mlei_E1_3_10252bp.bed', index = False, sep = "\t")

# Since centromere locations are not available for the examined genomes, we manually reviewed the E1, E2, and E3 eigenvectors
# for each chromosome to ensure that the signal accurately reflects compartmentalization patterns. Corrected eigenvalues were further
# used to visualize the comparmentalization preferences by using saddleplots.

eigenvector_track = bioframe.read_table('../compartmentalization/eigenvalues/mlei_E1_10252bp.bed')
eigenvector_track.columns = ['chrom','start','end','E1']

In [6]:
cvd = cooltools.expected_cis(
        clr=clr,
        view_df=view_df,
)

In [7]:
Q_LO = 0.025 # ignore 2.5% of genomic bins with the lowest E1 values
Q_HI = 0.975 # ignore 2.5% of genomic bins with the highest E1 values
N_GROUPS = 38 # divide remaining 95% of the genome into 38 equisized groups, 2.5% each
cvd

,region1,region2,dist,n_valid,count.sum,balanced.sum,count.avg,balanced.avg,balanced.avg.smoothed,balanced.avg.smoothed.agg
0,chr_1,chr_1,0,2009,NaN,NaN,NaN,NaN,NaN,NaN
1,chr_1,chr_1,1,1899,NaN,NaN,NaN,NaN,0.000266,0.000261
2,chr_1,chr_1,2,1862,515782.0,47.437650,277.004296,0.025477,0.023611,0.023088
3,chr_1,chr_1,3,1842,320855.0,30.486275,174.188382,0.016551,0.016164,0.016177
4,chr_1,chr_1,4,1832,236735.0,22.585958,129.222162,0.012329,0.012016,0.012246
...,...,...,...,...,...,...,...,...,...,...
40002,chr_13,chr_13,2271,2,9.0,0.001348,4.500000,0.000674,0.000041,0.000033
40003,chr_13,chr_13,2272,1,8.0,0.003774,8.000000,0.003774,0.000041,0.000033
40004,chr_13,chr_13,2273,0,0.0,0.000000,NaN,NaN,0.000041,0.000033
40005,chr_13,chr_13,2274,0,0.0,0.000000,NaN,NaN,0.000041,0.000033


In [ ]:
interaction_sum, interaction_count =  cooltools.saddle(
        clr,
        cvd,
        eigenvector_track,
        'cis',
        n_bins=N_GROUPS,
        qrange=(Q_LO,Q_HI),
        view_df=view_df
)

In [9]:
value_strength_b_a = interaction_sum/interaction_count
print(value_strength_b_a)
np.savetxt('../data/compartmentalization/20K_bins/mlei_saddle_strength_10252bp_B-A.tsv', value_strength_b_a, delimiter='\t')

[[2.26268559 1.66510578 1.59627419 ... 0.65071733 0.60621437 0.50639008]
 [1.66510578 1.50229456 1.41579889 ... 0.71651192 0.68655043 0.55671721]
 [1.59627419 1.41579889 1.34961224 ... 0.73191031 0.70594432 0.6037439 ]
 ...
 [0.65071733 0.71651192 0.73191031 ... 1.1230475  1.16449523 1.26776549]
 [0.60621437 0.68655043 0.70594432 ... 1.16449523 1.23497233 1.35192556]
 [0.50639008 0.55671721 0.6037439  ... 1.26776549 1.35192556 1.52470635]]


In [ ]:
saddleplot(eigenvector_track,
           interaction_sum/interaction_count,
           N_GROUPS,
           qrange=(Q_LO,Q_HI),
           cbar_kws={'label':'average observed/expected contact frequency'}
          );
plt.savefig('../data/compartmentalization/20K_bins/Mlei_saddle_plot_10252bpRes.pdf', transparent=True, bbox_inches='tight')